In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_parquet("data/processed/df.parquet")
df.head()

,customer_id,amount,gender,mcc_description,tr_description,day,time,time_category
0,39026145,-2245.92,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,0,10:23:26,рабочее время
1,39026145,56147.89,1.0,Финансовые институты — снятие наличности автом...,Взнос наличных через АТМ (в своем тер.банке),1,10:19:29,рабочее время
2,39026145,-56147.89,1.0,Денежные переводы,Списание с карты по операции “перевода с карты...,1,10:20:56,рабочее время
3,39026145,-1392.47,1.0,"Различные продовольственные магазины — рынки, ...",Покупка. POS ТУ СБ РФ,1,10:39:54,рабочее время
4,39026145,-920.83,1.0,"Различные продовольственные магазины — рынки, ...",Покупка. POS ТУ СБ РФ,2,15:33:42,рабочее время


#### Embeddings

In [3]:
import torch
from tqdm import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 512

In [4]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [5]:
model.to(DEVICE)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [6]:
def get_embeddings_batch(texts):
    inputs = tokenizer(
        texts, return_tensors="pt", padding=True, truncation=True, max_length=512
    )
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]  # CLS-токен
    return embeddings.cpu().numpy()


embeddings_list = []

for i in tqdm(range(0, len(df), BATCH_SIZE)):
    batch_texts = df["mcc_description"].iloc[i : i + BATCH_SIZE].tolist()
    batch_embeddings = get_embeddings_batch(batch_texts)
    embeddings_list.append(batch_embeddings)

# Объединяем все батчи
all_embeddings = np.vstack(embeddings_list)

# Добавляем в DataFrame (будет колонка с numpy-массивами)
df["embedding"] = list(all_embeddings)

100%|██████████| 13378/13378 [05:40<00:00, 39.31it/s]


In [7]:
df = df.drop(columns=["time", "mcc_description"], axis=1)
df = df.rename(columns={"embedding": "mcc_embeddings"})

In [8]:
df.head()

,customer_id,amount,gender,tr_description,day,time_category,mcc_embeddings
0,39026145,-2245.92,1.0,Оплата услуги. Банкоматы СБ РФ,0,рабочее время,"[-0.549412, 0.15420678, 0.6769444, -0.28600964..."
1,39026145,56147.89,1.0,Взнос наличных через АТМ (в своем тер.банке),1,рабочее время,"[0.29221445, -0.72320914, 0.47806624, -0.94505..."
2,39026145,-56147.89,1.0,Списание с карты по операции “перевода с карты...,1,рабочее время,"[-0.09898794, -0.3049472, -0.5152246, -0.52015..."
3,39026145,-1392.47,1.0,Покупка. POS ТУ СБ РФ,1,рабочее время,"[1.0021925, -0.20207441, -0.19583933, -1.30638..."
4,39026145,-920.83,1.0,Покупка. POS ТУ СБ РФ,2,рабочее время,"[1.0021925, -0.20207441, -0.19583933, -1.30638..."


In [9]:
df.isna().sum()

customer_id             0
amount                  0
gender            3118632
tr_description          0
day                     0
time_category           0
mcc_embeddings          0
dtype: int64

In [10]:
df.to_parquet("data/processed/df_for_agg.parquet", engine="pyarrow")